# Notebook Description

This notebook is used to fetch on chain data by The Graph API of UNISWAP V2. This is a draft so that something is useful and something is also useless. The formal version please refer to the PoolData_V2.
<br>Terminology, definition, and example please refer to https://docs.uniswap.org/protocol/V2/reference/API/entities#

# Config Environment

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.chdir("/content/gdrive/My Drive/DefiEconResearch/V2Data")

In [4]:
import pandas as pd
import numpy as np

# Query Function

In [5]:
import requests
from pprint import pprint

def run_query(q):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
                            '',
                            json={'query': q})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, q))

def run_query_var(q, var):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
                            '',
                            json={'query': q, 'variables': var})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, q))

def run_query_ether(q):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/blocklytics/ethereum-blocks'
                            '',
                            json={'query': q})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, q))

# Query the INSTANT overview of Uniswap V2: total liquidity, total volume (USD)

In [23]:
v2_overview_query = """
{
 uniswapFactory(id: "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f"){
   totalVolumeUSD
   totalLiquidityUSD
   totalVolumeETH
   totalLiquidityETH
   pairCount
   txCount
 }
}
"""
v2_overview = run_query(v2_overview_query)

pprint(v2_overview)

{'data': {'uniswapFactory': {'pairCount': 76439,
                             'totalLiquidityETH': '888302.4119904871118387331274559368',
                             'totalLiquidityUSD': '1635313051.682547404490780429959447',
                             'totalVolumeETH': '281571647.9369482253806664227052093',
                             'totalVolumeUSD': '413633328255.9816370214666716687079',
                             'txCount': '80453962'}}}


# (Delete)Query the HISTORICAL SNAPSHOT overview of Uniswap V2: total liquidity, total volume (USD)

In [141]:
# unix timestamp: GMT 2022/05/30 0:00:00  1653865200
# unix timestamp: GMT 2022/05/31 0:00:00  1653951600
timestamp_to_block_query = """
{
  blocks(first: 1, orderBy: timestamp, orderDirection: asc, where: {timestamp_gt: "1653951600"}) {
    id
    number
    timestamp
  }
}
"""
timestamp_to_block = run_query_ether(timestamp_to_block_query)
pprint(timestamp_to_block)

{'data': {'blocks': [{'id': '0x15a2e4c8ba92eb4cfa9f793d3ab15891ae5fe5c049f77abf8ca15df063cb9008',
                      'number': '14875301',
                      'timestamp': '1653951612'}]}}


# Get All Pairs

In [25]:
# # totalSupply: total supply of liquidity token distributed to LPs   unit as ETH
# get_paris_query_2 = """
# {
#   pairs(first: 10, orderBy: createdAtTimestamp, orderDirection: asc) 
#   { 
#      token0 {
#         id
#         symbol
#       }
#       token1 {
#         id
#         symbol
#       }
#      id
#      createdAtBlockNumber
#      createdAtTimestamp
#      totalSupply
#      txCount
#     }
# }
# """

# get_pairs_batch2 = run_query(get_paris_query_2)
# pprint(get_pairs_batch2)

{'data': {'pairs': [{'createdAtBlockNumber': '10008355',
                     'createdAtTimestamp': '1588710145',
                     'id': '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc',
                     'token0': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                                'symbol': 'USDC'},
                     'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                                'symbol': 'WETH'},
                     'totalSupply': '0.952643944936983285',
                     'txCount': '3543792'},
                    {'createdAtBlockNumber': '10008500',
                     'createdAtTimestamp': '1588712092',
                     'id': '0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee',
                     'token0': {'id': '0x8e870d67f660d95d5be530380d0ec0bd388289e1',
                                'symbol': 'PAX'},
                     'token1': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                                's

In [101]:
# Start from fetching the initial Batch 0: Query 1000 pairs on UNISWAP V2 order by created timestamp
get_paris_query_0 = """
{
  pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc) 
  { 
    token0 {
        id
        symbol
    }
    token1 {
        id
        symbol
    }
    id
    createdAtBlockNumber
    createdAtTimestamp
    totalSupply
    txCount
  }
}
"""

get_pairs_batch0 = run_query(get_paris_query_0)


In [102]:
# Create a dataframe to store the pairs info, initializa with the first 1000 pairs
df_all_pairs = pd.DataFrame.from_dict(get_pairs_batch0['data']['pairs'])

In [105]:
# Do iteration to fetch all the pairs, skip the first 1000 in batch 0
# Start from the last timestamp which is got by batch 0

iter_count = 0
pairs_iter = get_pairs_batch0['data']['pairs']  # Pair info list in each batch
total_pair_amount = len(pairs_iter)  # Initial the total pair amount by 1000 (in batch 0)

# Do loop until 
while len(pairs_iter) == 1000:
  # The last timestamp in the previous query batch
  last_gt = int(pairs_iter[-1]['createdAtTimestamp'])
  params_gt = {"createdAtTimestamp_gt": last_gt}

  # Query 1000 new pairs from the last timestamp
  query_iter = """
  query pairs($createdAtTimestamp_gt: Int!) {
    pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc
    where: {createdAtTimestamp_gt: $createdAtTimestamp_gt }) 
    { 
      token0 {
        id
        symbol
      }
      token1 {
        id
        symbol
      }
      id
      createdAtBlockNumber
      createdAtTimestamp
      totalSupply
      txCount
    }
  }
  """
  result_iter = run_query_var(query_iter, params_gt)

  # List of pair info for this batch
  pairs_iter = result_iter['data']['pairs']

  # Add list of this batch to the dataframe
  df_all_pairs = df_all_pairs.append(pairs_iter, ignore_index=True)

  # Summary for this batch, and update iterator
  iter_count = iter_count + 1
  total_pair_amount = total_pair_amount+len(pairs_iter)
  print('Batch: ', iter_count, '  Length of pair: ', len(pairs_iter), '  Total fected pairs: ', total_pair_amount)
  print('             Start from timestamp: ', last_gt)

print('---------------------------')
print('Amount of fetced pairs: ', total_pair_amount)


Batch:  1   Length of pair:  1000   Total fected pairs:  2000
             Start from timestamp:  1592097373
Batch:  2   Length of pair:  1000   Total fected pairs:  3000
             Start from timestamp:  1594041143
Batch:  3   Length of pair:  1000   Total fected pairs:  4000
             Start from timestamp:  1595937431
Batch:  4   Length of pair:  1000   Total fected pairs:  5000
             Start from timestamp:  1596802583
Batch:  5   Length of pair:  1000   Total fected pairs:  6000
             Start from timestamp:  1597709234
Batch:  6   Length of pair:  1000   Total fected pairs:  7000
             Start from timestamp:  1598348473
Batch:  7   Length of pair:  1000   Total fected pairs:  8000
             Start from timestamp:  1599034094
Batch:  8   Length of pair:  1000   Total fected pairs:  9000
             Start from timestamp:  1599508406
Batch:  9   Length of pair:  1000   Total fected pairs:  10000
             Start from timestamp:  1600106758
Batch:  10   Lengt

In [106]:
df_all_pairs

,token0,token1,id,createdAtBlockNumber,createdAtTimestamp,totalSupply,txCount
0,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,10008355,1588710145,0.952645034840461313,3545177
1,{'id': '0x8e870d67f660d95d5be530380d0ec0bd3882...,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee,10008500,1588712092,0.000000006379783,28
2,{'id': '0x06af07097c9eeb7fd685c692751d5c66db49...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x12ede161c702d1494612d19f05992f43aa6a26fb,10014178,1588788074,14.429278541823799433,825
3,{'id': '0x6b175474e89094c44da98b954eedeac49527...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,10042267,1589164213,120134.434231707494879581,1383828
4,{'id': '0x408e41876cccdc0f92210600ef5037265605...,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x07f068ca326a469fc1d87d85d448990c8cba7df9,10060679,1589411030,0.000360555127545398,1450
...,...,...,...,...,...,...,...
100587,{'id': '0x65ded4c951e9a20cdbbf4b8a6536e5027d39...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x66bb48c0f79225d7b8fc612d81b8e54cb53f6ac8,14933384,1654789350,0.000799999999999,41
100588,{'id': '0x18d20a9fd597bf1bba87c26c3db6c24232ab...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x47a6df38e445a534674b942bb32fe4971150ac76,14933424,1654789860,7957.119709486762496381,55
100589,{'id': '0x3c3f0dd5846de6e6564722040a954aa89656...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x4fc883848e6aa45c72428ed6515f32ccc8efe7f1,14933450,1654790190,37.549966711037174032,59
100590,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,{'id': '0xedfb2f23233e4c7b02a87da7171319159938...,0x456b759687aaf8ed4cfcf4d22efeb9b1545dc609,14933656,1654793732,7632.553468499999999,43


In [107]:
# Write dataframe to csv
df_all_pairs.to_csv("All_Pairs_Info_V2_20220609_H19M11.csv")

# Get Mints/Burns Transactions

In [27]:
# # Mint: Add Liquidity to pool,  Burn: Remove Liquidity to the pool
# # See example of Add ETH and RND to pair: ETH/RND 0x5449bd1a97296125252db2d9cf23d5d6e30ca3c1
# mint_transaction_query = """
# {
#  mints(first: 30, where: { pair: "0x5449bd1a97296125252db2d9cf23d5d6e30ca3c1" }, orderBy: timestamp, orderDirection: desc) {
#    transaction {
#      id
#      timestamp
#    }
#    to
#    liquidity
#    amount0
#    amount1
#    amountUSD
#  }
# }
# """
# mint_transaction = run_query(mint_transaction_query)

# pprint(mint_transaction)

candidatePairs = [
 "0xa478c2975ab1ea89e8196811f51a7b7ade33eb11",
 "0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5"
]
params_candidate_pairs = {"candidatePairs": candidatePairs}
mint_transaction_query = """
query($candidatePairs: [String!]) {
 mints(first: 30, where: { pair_in: $candidatePairs }, orderBy: timestamp, orderDirection: desc) {
   transaction {
     id
     timestamp
   }
   to
   liquidity
   amount0
   amount1
   amountUSD
 }
}
"""
mint_transaction = run_query_var(mint_transaction_query, params_candidate_pairs)

pprint(mint_transaction)

{'data': {'mints': [{'amount0': '20917.612421378506229795',
                     'amount1': '11.996883604053411427',
                     'amountUSD': '42119.13928067889754061696505107397',
                     'liquidity': '298.420730358101761339',
                     'to': '0x6d9893fa101cd2b1f8d1a12de3189ff7b80fdc10',
                     'transaction': {'id': '0xa1eaff753ecc8cb534032a8decabe9374f8ea62c6d247a7f4ceeb3daa0a33f14',
                                     'timestamp': '1654583776'}},
                    {'amount0': '1431086.920205154521392811',
                     'amount1': '1432237.745324',
                     'amountUSD': '2864417.136023481484046488768673554',
                     'liquidity': '1.271718317255244619',
                     'to': '0x0a827de54f4c6c2ae24bf52c5a278a045300e224',
                     'transaction': {'id': '0x86fcd975478ab7a53e7b2f3b0a5afeb2be2ce2f107c3215590321e9d0c4dc617',
                                     'timestamp': '1654560221'}},
   

# Compute Volume

In [81]:
# query current USD price of ETH in Uniswap based on a weighted average of stablecoins.
eth_price_query = """
{
 bundle(id: "1") {
   ethPrice
 }
}
"""
eth_price = run_query(eth_price_query)

pprint(eth_price)

{'data': {'bundle': {'ethPrice': '1794.997177138197787236690233247948'}}}


In [89]:
# query current USD price of ETH in Uniswap based on a weighted average of stablecoins.
eth_price_pairs = [
 "0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc"
]
params_eth_price_pairs = {"eth_price_pairs": eth_price_pairs}
eth_price_byvolume_query = """
query($eth_price_pairs: [String!]) {
 swaps(first: 1, 
  where: { pair_in: $eth_price_pairs, timestamp_gt: 1654503528 }, orderBy: timestamp, orderDirection: asc) {
   transaction {
     id
     timestamp
   }
   amount0In
   amount1In
   amount0Out
   amount1Out
   amountUSD
 }
}
"""
eth_price_byvolume = run_query_var(eth_price_byvolume_query, params_eth_price_pairs)

pprint(eth_price_byvolume)

if float(eth_price_byvolume['data']['swaps'][0]['amount0In']) != 0:
  eth_price_byvolume_result = float(eth_price_byvolume['data']['swaps'][0]['amount0In'])/float(eth_price_byvolume['data']['swaps'][0]['amount1Out'])
else:
  eth_price_byvolume_result = float(eth_price_byvolume['data']['swaps'][0]['amount0Out'])/float(eth_price_byvolume['data']['swaps'][0]['amount1In'])

print('ETH PRICE IS: ', eth_price_byvolume_result)

{'data': {'swaps': [{'amount0In': '539.626088',
                     'amount0Out': '0',
                     'amount1In': '0',
                     'amount1Out': '0.284165115514204348',
                     'amountUSD': '538.676591762887036143561404921746',
                     'transaction': {'id': '0xea9d52361194f848356746a2182965d0419138ff170e67fea13a2803fc464fef',
                                     'timestamp': '1654503529'}}]}}
ETH PRICE IS:  1898.9878015939155


In [90]:
print(2634.665349/1.472669921217362735)
print(167.37499/0.093561282132463456)

print(1898.9878015939155*0.016)

1789.0399681838337
1788.9343346431656
30.383804825502647


In [12]:
# Type 1: trade against WETH
pair_daily_volume_query = """
{
swaps(orderBy: timestamp, orderDirection: desc, where:
 { pair: "0xc9919a9eb691f571e41db417f911135b4ac46fe7" }
) {
     transaction {
        id
        timestamp
      }
     pair {
       token0 {
         symbol
       }
       token1 {
         symbol
       }
     }
     id
     transaction
     amount0In
     amount0Out
     amount1In
     amount1Out
     amountUSD
     to
 }
}
"""
pair_daily_volume = run_query(pair_daily_volume_query)

pprint(pair_daily_volume)

{'data': {'swaps': [{'amount0In': '0',
                     'amount0Out': '954.10490882867114783',
                     'amount1In': '0.087109307299713778',
                     'amount1Out': '0',
                     'amountUSD': '162.3017323177359330773202940380697',
                     'id': '0xe976f2ebd70135153e8522c3aaa3c9d489717ef621d69b9e00a9f215d0474918-8',
                     'pair': {'token0': {'symbol': 'LSS'},
                              'token1': {'symbol': 'WETH'}},
                     'to': '0x62e3c242b5e903071458ad90a160493d84911c77',
                     'transaction': {'id': '0xe976f2ebd70135153e8522c3aaa3c9d489717ef621d69b9e00a9f215d0474918',
                                     'timestamp': '1654768688'}},
                    {'amount0In': '649.3506494',
                     'amount0Out': '0',
                     'amount1In': '0',
                     'amount1Out': '0.057907220009412727',
                     'amountUSD': '102.5315103855514293805263468787294',

# Get Popular Pools (We need order by volume)

## Top 50 Most liquidity pairs (order by liquidity) (Delete)

In [147]:
most_liquid_pairs_query = """
{
 pairs(first: 50, orderBy: reserveUSD, orderDirection: desc) {
   id
 }
}
"""
most_liquid_pairs = run_query(most_liquid_pairs_query)

pprint(most_liquid_pairs)

{'data': {'pairs': [{'id': '0xe6c78983b07a07e0523b57e18aa23d3ae2519e05'},
                    {'id': '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc'},
                    {'id': '0x21b8065d10f73ee2e260e5b47d3344d3ced7596e'},
                    {'id': '0x9928e4046d7c6513326ccea028cd3e7a91c7590a'},
                    {'id': '0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5'},
                    {'id': '0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852'},
                    {'id': '0xe1573b9d29e2183b1af0e743dc2754979a40d237'},
                    {'id': '0x7d7e813082ef6c143277c71786e5be626ec77b20'},
                    {'id': '0x61b62c5d56ccd158a38367ef2f539668a06356ab'},
                    {'id': '0xccb63225a7b19dcf66717e4d40c9a72b39331d61'},
                    {'id': '0x0617d5ffb29c03ac35f1863b8a50ce1b52d446f6'},
                    {'id': '0x9816f26f43c4c02df0daae1a0ba6a4dcd30b8ab7'},
                    {'id': '0x9c4fe5ffd9a9fc5678cfbd93aa2d4fd684b67c4c'},
                    {'id': '0x3c92befe

The result is consistent with: https://v2.info.uniswap.org/pairs
<br>Note that it needs to cancel the click of `Hide untracked pairs`
<br>Reasons refered from Uniswap: USD amounts may be inaccurate in low liquidity pairs or pairs without ETH or stable coins. 
<br>Example: the top liquid pair (UETH/ULCK: '0xe6c78983b07a07e0523b57e18aa23d3ae2519e05') is invisible if clicking the hide button.

## Top 50 most volume pairs 

In [5]:
# Take example of 2022/06/05 0:00:00 (GMT+0)  timestamp: 1654387200
most_volume_pairs_query = """
{
 pairDayDatas(first: 50, orderBy: dailyVolumeUSD, orderDirection: desc,
   where: {
     date: 1654387200
   }
 ) {
     pairAddress
     token0
     token1
     reserve0
     reserve1
     reserveUSD
     totalSupply
     dailyTxns
     dailyVolumeToken0
     dailyVolumeToken1
     dailyVolumeUSD
 }
}
"""
most_volume_pairs = run_query(most_volume_pairs_query)

pprint(most_volume_pairs)

{'data': {'pairDayDatas': [{'dailyTxns': '66',
                            'dailyVolumeToken0': '75.590458431395684271',
                            'dailyVolumeToken1': '20391.289551617179340048',
                            'dailyVolumeUSD': '18044577.16616131095138411684391161',
                            'pairAddress': '0x26b1c843b41ab3ac6146f859d31dcb2031f5992b',
                            'reserve0': '1.767872165974808497',
                            'reserve1': '0.103575127241738334',
                            'reserveUSD': '185.007627282511291050410476253084',
                            'totalSupply': '0.397489622506046319'},
                           {'dailyTxns': '568',
                            'dailyVolumeToken0': '1419137291173.507923209293317101',
                            'dailyVolumeToken1': '13969.023979241586925419',
                            'dailyVolumeUSD': '12330161.8603183632501027905931409',
                            'pairAddress': '0x9112d1761124

In [6]:
# Create a dataframe to store the popular pairs by daily volume
df_most_volume_pairs = pd.DataFrame.from_dict(most_volume_pairs['data']['pairDayDatas'])
df_most_volume_pairs

,pairAddress,reserve0,reserve1,reserveUSD,totalSupply,dailyTxns,dailyVolumeToken0,dailyVolumeToken1,dailyVolumeUSD
0,0x26b1c843b41ab3ac6146f859d31dcb2031f5992b,1.767872165974808497,0.103575127241738334,185.007627282511291050410476253084,0.397489622506046319,66,75.590458431395684271,20391.289551617179340048,18044577.16616131095138411684391161
1,0x9112d1761124d4ec7aca02014d6e215970f87e33,16350613868.3869050774532976,0.519906171224847849,939.5606044999527414057226528192348,76481.596984279553163924,568,1419137291173.507923209293317101,13969.023979241586925419,12330161.8603183632501027905931409
2,0x7a809081f991ecfe0ab2727c7e90d2ad7c2e411e,7278284.442263,53711361613298.579569450654648315,14562523.79614463010296044124735658,17000.372001445604539439,1130,9352341.202544,70352106872638.631096732439854899,9356068.767914834418701521183459145
3,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,79053219.45668,43762.070870754729340122,158171118.2947124454132655087211276,0.949291097224270471,2352,7944060.293021,4416.753923820066502526,7946290.543183526065581561379215337
4,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,17150.6216251843007197,31025675.01265,61986741.93124518014117041922362383,0.358735792422925442,988,1985.069246836202839412,3568632.914985,3567514.616175831645067170006679848
5,0x6033368e4a402605294c91cf5c03d72bd96e7d8d,12923069.47957958366364951,1700.778808333284489473,6150046.731936887471116457848148126,122495.841175114799947429,339,13267781.010060489687022394,1890.06115884505521772,3406788.640697228554984176713442879
6,0x5fc5b2a2a84e577e8eb3ddd00d6b18cdfc976645,1.405332611560299561,4879.509395835,2540.637390630996495380319988943423,0.000000001693876534,52,2370.283509828503112744,29395.18460405,2117245.547362322563131098884959673
7,0xc9919a9eb691f571e41db417f911135b4ac46fe7,19492.596085670694340666,1.743545235299297085,6230.195069012601572355339755270908,134.786530428602600527,5,39890.453065187663700397,2315.4491801142244733,2068207.090312683960461532632688075
8,0x55d5c232d921b9eaa6b37b5845e439acd04b4dba,9561300.85832576,262.860789083706771544,950046.2579647230507226365979986013,0.141939588833336829,1084,31687521.75123634,921.849962520731810018,1656077.459784708517237385640827616
9,0x5281e311734869c64ca60ef047fd87759397efe6,1590.131078037393753415,194274555743.182871721195811459,5747440.852724872320382470833962709,13027895.889162734975983901,354,693.44156563152035369,84402864787.057603459657130526,1242412.424525914084718022581442596


In [8]:
from tqdm import tqdm

# Create a data frame of orders
df_top50_pairs_info = pd.DataFrame({'pairAddress': df_most_volume_pairs['pairAddress'], 'dailyVolumeUSD': df_most_volume_pairs['dailyVolumeUSD'], 'token0_symbol': np.nan, 'token0_id': np.nan, 'dailyVolumeToken0': df_most_volume_pairs['dailyVolumeToken0'], 'reserve0': df_most_volume_pairs['reserve0'], 'token1_symbol': np.nan, 'token1_id': np.nan, 'dailyVolumeToken1': df_most_volume_pairs['dailyVolumeToken1'], 'reserve1': df_most_volume_pairs['reserve1'], 'reserveUSD': df_most_volume_pairs['reserveUSD'], 'totalSupply': df_most_volume_pairs['totalSupply'], 'dailyTxns': df_most_volume_pairs['dailyTxns'], }, index=df_most_volume_pairs.index)

for index, row in tqdm(df_most_volume_pairs.iterrows(), total=df_most_volume_pairs.shape[0]):
  this_id = row['pairAddress']  # Pair address of this row
  params_pair_id = {"id": this_id}
  pair_info_query = """
    query pair($id: String!) 
    {
      pair(id: $id){
      token0 {
        id
        symbol
      }
      token1 {
        id
        symbol
      }
      }
    }
  """
  pair_info = run_query_var(pair_info_query, params_pair_id)
  df_top50_pairs_info.loc[index] = [row['pairAddress'], row['dailyVolumeUSD'], pair_info['data']['pair']['token0']['symbol'], pair_info['data']['pair']['token0']['id'], row['dailyVolumeToken0'], row['reserve0'], pair_info['data']['pair']['token1']['symbol'], pair_info['data']['pair']['token0']['id'], row['dailyVolumeToken1'], row['reserve1'], row['reserveUSD'], row['totalSupply'], row['dailyTxns']]

100%|██████████| 50/50 [00:10<00:00,  4.78it/s]


In [9]:
df_top50_pairs_info

,pairAddress,dailyVolumeUSD,token0_symbol,token0_id,dailyVolumeToken0,reserve0,token1_symbol,token1_id,dailyVolumeToken1,reserve1,reserveUSD,totalSupply,dailyTxns
0,0x26b1c843b41ab3ac6146f859d31dcb2031f5992b,18044577.16616131095138411684391161,sILV2,0x7e77dcb127f99ece88230a64db8d595f31f1b068,75.590458431395684271,1.767872165974808497,WETH,0x7e77dcb127f99ece88230a64db8d595f31f1b068,20391.289551617179340048,0.103575127241738334,185.007627282511291050410476253084,0.397489622506046319,66
1,0x9112d1761124d4ec7aca02014d6e215970f87e33,12330161.8603183632501027905931409,SHIT,0x4e4a47cac6a28a62dcc20990ed2cda9bc659469f,1419137291173.507923209293317101,16350613868.3869050774532976,WETH,0x4e4a47cac6a28a62dcc20990ed2cda9bc659469f,13969.023979241586925419,0.519906171224847849,939.5606044999527414057226528192348,76481.596984279553163924,568
2,0x7a809081f991ecfe0ab2727c7e90d2ad7c2e411e,9356068.767914834418701521183459145,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,9352341.202544,7278284.442263,CAW,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,70352106872638.631096732439854899,53711361613298.579569450654648315,14562523.79614463010296044124735658,17000.372001445604539439,1130
3,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,7946290.543183526065581561379215337,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,7944060.293021,79053219.45668,WETH,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,4416.753923820066502526,43762.070870754729340122,158171118.2947124454132655087211276,0.949291097224270471,2352
4,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,3567514.616175831645067170006679848,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1985.069246836202839412,17150.6216251843007197,USDT,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3568632.914985,31025675.01265,61986741.93124518014117041922362383,0.358735792422925442,988
5,0x6033368e4a402605294c91cf5c03d72bd96e7d8d,3406788.640697228554984176713442879,X2Y2,0x1e4ede388cbc9f4b5c79681b7f94d36a11abebc9,13267781.010060489687022394,12923069.47957958366364951,WETH,0x1e4ede388cbc9f4b5c79681b7f94d36a11abebc9,1890.06115884505521772,1700.778808333284489473,6150046.731936887471116457848148126,122495.841175114799947429,339
6,0x5fc5b2a2a84e577e8eb3ddd00d6b18cdfc976645,2117245.547362322563131098884959673,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2370.283509828503112744,1.405332611560299561,GIZA,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,29395.18460405,4879.509395835,2540.637390630996495380319988943423,0.000000001693876534,52
7,0xc9919a9eb691f571e41db417f911135b4ac46fe7,2068207.090312683960461532632688075,LSS,0x3b9be07d622accaed78f479bc0edabfd6397e320,39890.453065187663700397,19492.596085670694340666,WETH,0x3b9be07d622accaed78f479bc0edabfd6397e320,2315.4491801142244733,1.743545235299297085,6230.195069012601572355339755270908,134.786530428602600527,5
8,0x55d5c232d921b9eaa6b37b5845e439acd04b4dba,1656077.459784708517237385640827616,HEX,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,31687521.75123634,9561300.85832576,WETH,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,921.849962520731810018,262.860789083706771544,950046.2579647230507226365979986013,0.141939588833336829,1084
9,0x5281e311734869c64ca60ef047fd87759397efe6,1242412.424525914084718022581442596,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,693.44156563152035369,1590.131078037393753415,CULT,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,84402864787.057603459657130526,194274555743.182871721195811459,5747440.852724872320382470833962709,13027895.889162734975983901,354


In [10]:
# Write dataframe to csv
df_top50_pairs_info.to_csv("Top_50_Volume_Pairs_20220605.csv")

# Compute Directional Volume

In [100]:
# Firstly, fetch all transaction in each day
directional_volume_pair = [
 "0xc9919a9eb691f571e41db417f911135b4ac46fe7"
]
params_directional_volume_pair = {"directional_volume_pair": directional_volume_pair}

swaps_transaction_query = """
query($directional_volume_pair: [String!]) {
 swaps(first: 5, where: { pair_in: $directional_volume_pair, timestamp_gt: 1654387200}, orderBy: timestamp, orderDirection: asc) {
   transaction {
     id
     timestamp
   }
   amount0In
   amount1In
   amount0Out
   amount1Out
   amountUSD
 }
}
"""
swaps_transaction = run_query_var(swaps_transaction_query, params_directional_volume_pair)

pprint(swaps_transaction)

{'data': {'swaps': [{'amount0In': '874.358028061008428415',
                     'amount0Out': '0',
                     'amount1In': '0',
                     'amount1Out': '0.080380204908462933',
                     'amountUSD': '140.9912911183920638860407598128405',
                     'transaction': {'id': '0x6a8031402df1401112a7b5a17654673c708d111912946ac1beb169a13d506f59',
                                     'timestamp': '1654417560'}},
                    {'amount0In': '0',
                     'amount0Out': '384.915147460101858337',
                     'amount1In': '0.03469375',
                     'amount1Out': '0',
                     'amountUSD': '62.32797447381097606245121727947374',
                     'transaction': {'id': '0xa3a509d48b4c9c6432ffd0c46cac6e6d1fad5395a4cf1dcaa0fb6dd98d1a7b82',
                                     'timestamp': '1654426696'}},
                    {'amount0In': '0',
                     'amount0Out': '19167.809153296665628187',
        

# Pair Daily Aggregated

In [97]:
# Example: Pair: DAI/WETH: 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11
# First 10 day from given timestamp: 1653001200  (2022/5/20 GMT 0:00:00)
pair_daily_aggregated_query = """
{
 pairDayDatas(first: 1, orderBy: date, orderDirection: asc,
   where: {
     pairAddress: "0xc9919a9eb691f571e41db417f911135b4ac46fe7",
     date: 1654387200
   }
 ) {
     date
     dailyVolumeToken0
     dailyVolumeToken1
     dailyVolumeUSD
     reserveUSD
 }
}
"""
pair_daily_aggregated = run_query(pair_daily_aggregated_query)

pprint(pair_daily_aggregated)

{'data': {'pairDayDatas': [{'dailyVolumeToken0': '39890.453065187663700397',
                            'dailyVolumeToken1': '2315.4491801142244733',
                            'dailyVolumeUSD': '2068207.090312683960461532632688075',
                            'date': 1654387200,
                            'reserveUSD': '6230.195069012601572355339755270908'}]}}


The result is consist with: https://v2.info.uniswap.org/pair/0xa478c2975ab1ea89e8196811f51a7b7ade33eb11
<br>For example, the volume of DAI/ETH at 2022/05/29 (timestamp: 1653782400) is USD $509629

Next to do: confirm with the calculation from fetching transactions.

# Token Overview

Token data can be fetched using the token contract address as an ID. Token data is aggregated across all pairs the token is included in. Any token that is included in some pair in Uniswap can be queried.
<br>reference: https://docs.uniswap.org/protocol/V2/reference/API/queries

In [150]:
# Example: The allPairs field gets the first 200 pairs DAI is included in sorted by liquidity in derived USD
token_overview_query = """
{
 token(id: "0x6b175474e89094c44da98b954eedeac495271d0f"){
   name
   symbol
   decimals
   derivedETH
   tradeVolumeUSD
   totalLiquidity
 }
}
"""
token_overview = run_query(token_overview_query)

pprint(token_overview)

{'data': {'token': {'decimals': '18',
                    'derivedETH': '0.0005489257369435742850409519739633295',
                    'name': 'Dai Stablecoin',
                    'symbol': 'DAI',
                    'totalLiquidity': '71259638.947566970658950017',
                    'tradeVolumeUSD': '19170068979.52979291004214829078219'}}}


The allPairs field gets the first 200 pairs DAI is included in sorted by liquidity in derived USD.
<br>The result of `total liquidity` is consistent with https://v2.info.uniswap.org/token/0x6b175474e89094c44da98b954eedeac495271d0f
<br><br>TODO: `trade volumn` is not `volume (24hrs). We need to calculate the the daily volume. 

# All Token in Uniswap V2

In [8]:
# Start from fetching the initial Batch 0: Query 1000 tokens on UNISWAP V2 order by id (asc)
# tradeVolumeUSD: amount of token in USD traded all time across pairs (only for tokens with liquidity above minimum threshold)
# untrackedVolumeUSD: amount of token in USD traded all time across pairs (no minimum liquidity threshold)
get_all_tokens_batch0_query = """
{
   tokens(first: 1000, orderBy: id, orderDirection: asc) {
     id
     name
     symbol
     tradeVolume
     tradeVolumeUSD
     untrackedVolumeUSD
     txCount
     totalLiquidity
     derivedETH
   }
}
"""
get_all_tokens_batch0 = run_query(get_all_tokens_batch0_query)

In [9]:
# Create a dataframe to store the tokens info, initializa with the first 1000 tokens
df_all_tokens = pd.DataFrame.from_dict(get_all_tokens_batch0['data']['tokens'])

In [10]:
# Do iteration to fetch all the tokens, skip the first 1000 in batch 0
# Start from the last id which is got by batch 0

tokens_iter_count = 0
tokens_iter = get_all_tokens_batch0['data']['tokens']  # Token info list in each batch
total_tokens_amount = len(tokens_iter)  # Initial the total token amount by 1000 (in batch 0)

# Do loop until 
while len(tokens_iter) == 1000:
  # The last id in the previous query batch
  last_id_gt = str(tokens_iter[-1]['id'])
  params_id_gt = {"id_in": last_id_gt}

  # Query 1000 new tokens from the last id
  query_iter = """
  query tokens($id_in: String!) {
    tokens(first: 1000, orderBy: id, orderDirection: asc
    where: {id_gt: $id_in }) 
    { 
      id
      name
      symbol
      tradeVolume
      tradeVolumeUSD
      untrackedVolumeUSD
      txCount
      totalLiquidity
      derivedETH
    }
  }
  """
  tokens_result_iter = run_query_var(query_iter, params_id_gt)

  # List of token info for this batch
  tokens_iter = tokens_result_iter['data']['tokens']

  # Add list of this batch to the dataframe
  df_all_tokens = df_all_tokens.append(tokens_iter, ignore_index=True)

  # Summary for this batch, and update iterator
  tokens_iter_count = tokens_iter_count + 1
  total_tokens_amount = total_tokens_amount+len(tokens_iter)
  print('Batch: ', tokens_iter_count, '  Length of pair: ', len(tokens_iter), '  Total fected pairs: ', total_tokens_amount)
  print('             Start from id: ', last_id_gt)

print('---------------------------')
print('Amount of fetced pairs: ', total_tokens_amount)


Batch:  1   Length of pair:  1000   Total fected pairs:  2000
             Start from id:  0x03703a5e60918059baf5481c5d55a553890207a2
Batch:  2   Length of pair:  1000   Total fected pairs:  3000
             Start from id:  0x06f3ef6005c1e245a697fd4c9458aae95aaf8f29
Batch:  3   Length of pair:  1000   Total fected pairs:  4000
             Start from id:  0x0a92ca117788a18f6b38cf0261122a9ea95b7e4b
Batch:  4   Length of pair:  1000   Total fected pairs:  5000
             Start from id:  0x0e596b8a86082e8b0b1ae0aa8447df38c7851941
Batch:  5   Length of pair:  1000   Total fected pairs:  6000
             Start from id:  0x1208168aa6f7bf67f79ba922678f3940498c6149
Batch:  6   Length of pair:  1000   Total fected pairs:  7000
             Start from id:  0x158009a14f413065a5734b6c3d8261cfa45a0742
Batch:  7   Length of pair:  1000   Total fected pairs:  8000
             Start from id:  0x19093953fd8e4ea72d1a7dd5b7b72804e69923dc
Batch:  8   Length of pair:  1000   Total fected pairs:  9000


In [11]:
df_all_tokens

,id,name,symbol,tradeVolume,tradeVolumeUSD,untrackedVolumeUSD,txCount,totalLiquidity,derivedETH
0,0x00000000000045166c45af0fc6e4cf31d9e14b9a,TopBidder,BID,36520506.79045913892332785,23362640.36887859622558915387398381,23466279.70596361174072586983085042,2706,2252392.839917663338975365,0.00010395165265860908001061772755218
1,0x0000000000004946c0e9f43f4dee607b0ef1fa1c,Chi Gastoken by 1inch,CHI,43580282,19360715.70503782693540014871432548,65455773.59876190292325552554250769,75957,23374,0
2,0x000000000000d0151e748d25b766e77efe2a6c83,XDEFI Governance Token,XDEX,63890993.176575438849139776,18713723.91429208465798213093796374,18977868.34193645649430618371779554,4400,7459590.572861402421977784,0.00001160462895444170070268104491209126
3,0x0000000000085d4780b73119b644ae5ecd22b376,TrueUSD,TUSD,184555032.957792364526045589,176239741.7006048194469590153693985,185840599.5609657622640178364773038,47058,47430.468955485871691916,0.0005483537645577140176002674018884014
4,0x0000000000095413afc295d19edeb1ad7b71c952,Tokenlon,LON,142714234.348646246093382695,518202509.2165818130891579209118188,520054187.7708393702893219636096705,57980,1935283.541694594656834851,0.000434962232487259996007679151188112
...,...,...,...,...,...,...,...,...,...
71269,0xfffed8edc3ddc66de57f7fea41f07f98595f0a5c,TradeStars,TSX,0,0,0,2,0.00000000000013239,0.00006042752473751793942140645063826573
71270,0xffff49dc8ed20043fb9e5d65ea2f3e071958df8e,Alaskan Malamute,MALA,21789078739.712294975,0,5393.769723343128401960611013309801,7,0.000002959,0.0000000001145657316661034133153092260898952
71271,0xfffff4ff386013f4b9dac8874f856dec0e802467,Moon Doge,MOGE,19150445234.85529503422301082,0,1173.552612430322387524868188060823,8,0.000000000180785101,0.000000005531429274141346415488077195033898
71272,0xffffffff2ba8f66d4e51811c5190992176930278,Furucombo,COMBO,130417880.769097013663764675,185309346.3696076804844877325321709,185516831.2283832158632967962866492,23842,2061453.687712022160325331,0.00002902950409227019807739813744618048


In [12]:
# Write dataframe to csv
df_all_tokens.to_csv("All_Tokens_Info_V2_20220610.csv")

# Get top 200 popular tokens (order by the total trade volume USD)

In [163]:
# Get the top 200 popular tokens order by the total trade volume USD
get_popular_tokens_query = """
{
   tokens(first: 200, orderBy: tradeVolumeUSD, orderDirection: desc) {
     id
     name
     symbol
     tradeVolumeUSD
   }
}
"""
get_popular_tokens = run_query(get_popular_tokens_query)

In [164]:
# Create a dataframe to store the popular tokens
df_popular_tokens = pd.DataFrame.from_dict(get_popular_tokens['data']['tokens'])
df_popular_tokens

,id,name,symbol,tradeVolumeUSD
0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,387545039528.004554724840496930448
1,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD//C,USDC,57024192803.24888387524429033930168
2,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,46596921351.57798248473591458094727
3,0x6b175474e89094c44da98b954eedeac495271d0f,Dai Stablecoin,DAI,19170069015.5647184759042331426974
4,0x05934eba98486693aaec2d00b0e9ce918e37dc3f,You don't blacklist delta.financial,SCAMMY,11029715648.72159798743878461135205
...,...,...,...,...
195,0xeec2be5c91ae7f8a338e1e5f3b5de49d07afdc81,Dopex Governance Token,DPX,287878518.4337276001149145019920483
196,0x0d8775f648430679a709e98d2b0cb6250d2887ef,Basic Attention Token,BAT,287655138.7316011364479310256153892
197,0x3155ba85d5f96b2d030a4966af206230e46849cb,THORChain ETH.RUNE,RUNE,287355918.9337234532037635607585532
198,0x7bef710a5759d197ec0bf621c3df802c2d60d848,SPLYT SHOPX,SHOPX,286422097.6157676736985608599418268


In [165]:
# Write dataframe to csv
df_popular_tokens.to_csv("Popular_Tokens_V2_20220605.csv")

# Token Daily Aggregated

In [166]:
token_address_lower = str.lower('0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2')
print(token_address_lower)

0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


In [167]:
# Example of WETH: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2    lower: 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
token_daily_aggregated_query = """
{
 tokenDayDatas(orderBy: date, orderDirection: desc,
  where: {
    token: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"
  }
 ) {
    id
    date
    priceUSD
    totalLiquidityToken
    totalLiquidityUSD
    totalLiquidityETH
    dailyVolumeETH
    dailyVolumeToken
    dailyVolumeUSD
 }
}
"""
token_daily_aggregated= run_query(token_daily_aggregated_query)

pprint(token_daily_aggregated)

{'data': {'tokenDayDatas': [{'dailyVolumeETH': '299863.6118280715907828300597220015',
                             'dailyVolumeToken': '62193.703571700040825615',
                             'dailyVolumeUSD': '111358618.0086406708825746276440674',
                             'date': 1654387200,
                             'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-19148',
                             'priceUSD': '1820.991499255286792995784432639022',
                             'totalLiquidityETH': '344758.186848516584138967',
                             'totalLiquidityToken': '344758.186848516584138967',
                             'totalLiquidityUSD': '627801727.549814512357176365853776'},
                            {'dailyVolumeETH': '23781.19905333587039978189993302351',
                             'dailyVolumeToken': '29865.287292324909820506',
                             'dailyVolumeUSD': '52942529.95620032390325585129382752',
                             'date': 

EXAMPLE: 2022/06/01 timestamp: 1654041600
<br>DAI
<br>
{'dailyVolumeETH': '13052405201.72978201978725444517328',
                             'dailyVolumeToken': '4134944.732638395018173245',
                             **'dailyVolumeUSD': '4136483.377571504628199639668034694'**,
                             'date': 1654041600,
                             'id': '0x6b175474e89094c44da98b954eedeac495271d0f-19144',
                             'priceUSD': '1.000484172329567020191753518876482',
                             'totalLiquidityETH': '36227.35302188222791706482919363885',
                             'totalLiquidityToken': '65963489.599730641577233643',
                             **'totalLiquidityUSD': '65995427.29615651307026032696281615'**},

ETH
<br>
{'dailyVolumeETH': '34412.09803371055370364681041352234',
                             'dailyVolumeToken': '43803.417736487049312112',
                             **'dailyVolumeUSD': '82393163.32937788713396196558889604'**,
                             'date': 1654041600,
                             'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-19144',
                             'priceUSD': '1821.715671919287824651620977197333',
                             'totalLiquidityETH': '343608.778666151955888966',
                             'totalLiquidityToken': '343608.778666151955888966',
                             **'totalLiquidityUSD': '625957497.1051748619814372402967176'**},

The result is consistent with https://v2.info.uniswap.org/token/0x6b175474e89094c44da98b954eedeac495271d0f
<br> Note: it is daily aggregated, not 24hrs. This is what we need. 

TODO: MANUALLY COMPUTE THE VOLUME from transactions (**directional**)

# Get Transactions (Delete, this method get union, not intersection of pairs).

In [168]:
# UNI/ETH: 0xd3d2e2692501a5c9ca623199d38826e513033a17
# ETH/DAI: 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11
allPairs = [
 "0xd3d2e2692501a5c9ca623199d38826e513033a17",
 "0xa478c2975ab1ea89e8196811f51a7b7ade33eb11"
]

params_all_pair = {"allPairs": allPairs}

get_transactions_query = """
query($allPairs: [String!]) {
 swaps(first: 30, where: { pair_in: $allPairs }, orderBy: timestamp, orderDirection: desc) {
   transaction {
     id
     timestamp
   }
   amount0In
   amount0Out
   amount1In
   amount1Out
   amountUSD
   to
 }
}
"""
get_transactions = run_query_var(get_transactions_query, params_all_pair)

pprint(get_transactions)

{'data': {'swaps': [{'amount0In': '0',
                     'amount0Out': '36.007498224129839742',
                     'amount1In': '0.019825',
                     'amount1Out': '0',
                     'amountUSD': '36.03492556586208485191520893313049',
                     'to': '0x74de5d4fcbf63e00296fd95d33236b9794016631',
                     'transaction': {'id': '0xf7ac34f2bf88f65b3ce558716d60e4d2cc7fe4cf505fd3017500d7262e72aac7',
                                     'timestamp': '1654461033'}},
                    {'amount0In': '0',
                     'amount0Out': '354.931347289669836673',
                     'amount1In': '0.195409138876524511',
                     'amount1Out': '0',
                     'amountUSD': '355.1312871180965740396489984356085',
                     'to': '0xec54859519293b8784bc5bf28144166f313618af',
                     'transaction': {'id': '0x53980dfd18ecad4c60345322cb84e2b2c34f25b152126977550c126b37ab788c',
                                 

# TODO